In [ ]:
print("PostgreSQL + Python: lesson 2. Main data types of PostgreSQL and simply query.")

До початку роботи з матеріалом лекції прошу Вас впевнитись що контейнери з PostgreSQL і pgAdmin - запущені і працюють, і обраний Вами клієнтський застосунок для взаємодії з БД (DBeaver|pgAdmin) підключений і працює.

До того ж нам необхідно імпортувати створені на попередній лекції функції для роботи з БД з нашого коду - для цього вони перенесені в модуль __python_postgrsql.py__. Зробимо це:

In [ ]:
from python_postgresql import execute_query, execute_read_query, connection

Тепер ми без перешкод можемо перейти до вивчення матеріалу.

### Основні типи даних PostgreSQL

На відмінність від SQLite - з якою ми знайомились раніше - PostgreSQL підтримує значно більший набір типів даних. Це дозволяє оптимально підбирати типи під існуючи задачі і перекласти на рівень БД контроль форматів дани. Більше того - PostgreSQL надає можливість створоювати користувацькі типи даних використовуючи команду CREATE TYPE ([докладно тут](https://www.postgresql.org/docs/14/sql-createtype.html)).
Ми не будемо докладно вивчати можливість створення своїх типів даних в рамках нашого курсу, а стисло пройдемось по основних вбудованих типах даних які ми будемо використовувати протягом курск і їх особливостях. Значна частина типів даних залишиться за межами нашого короткого курсу, при необхідності ви можете докладно [познайомитись з ними тут](https://www.postgresql.org/docs/14/datatype.html).

1. __Числові__

    Ім'я  | Розмір	| Опис | Діапазон
    ------|---------|------|----------
    smallint | 2 байти | ціле в невеликому діапазоні | -32768 .. +32767
    integer | 4 байти | типовий вибір для цілих чисел | -2147483648.. +2147483647
    bigint | 8 байт  | ціле у великому діапазоні | -9223372036854775808 .. 9223372036854775807
    decimal | змінний	| дійсне число із зазначеною точністю | до 131072 цифр до десяткової точки і до 16383 - після
    numeric | змінний | дійсне число із зазначеною точністю | до 131072 цифр до десяткової точки і до 16383 - після
    real | 4 байти | дійсне число зі змінною точністю | точність у межах 6 десяткових цифр
    double precision | 8 байт | дійсне число зі змінною точністю | точність у межах 15 десяткових цифр
    smallserial | 2 байти | невелике ціле з автозбільшенням | 1 .. 32767
    serial | 4 байти | ціле з автозбільшенням | 1 .. 2147483647
    bigserial | 8 байт | велике ціле з автозбільшенням | 1 .. 9223372036854775807


    1. [Цілі числа - smallint, integer, bigint](https://www.postgresql.org/docs/14/datatype-numeric.html#DATATYPE-INT)
        Типи зберігають цілі числа - тобто числа без дробової частини, що мають різні допустимі діапазони. Спроба зберегти значення, що виходить за межі діапазону, призведе до помилки.
    2. [Числа з довільною точністю - numeric](https://www.postgresql.org/docs/14/datatype-numeric.html#DATATYPE-NUMERIC-DECIMAL).
        Тип numeric дозволяє зберігати цифри з дуже великою кількістю цифр. Він особливо рекомендується для зберігання грошових сум та інших величин, де важлива точність. Обчислення з типом numeric дають точні результати, де це можливо, наприклад, при додаванні, відніманні та множенні. Однак операції зі значеннями numeric виконуються набагато повільніше, ніж з цілими числами або з типами плаваючою точкою, описаними далі.
        Далі ми використовуємо такі терміни: scale значення numeric визначає кількість десяткових цифр у дрібній частині, праворуч від десяткової точки, а precision — загальна кількість значущих цифр у числі, тобто кількість цифр з обох боків десяткової точки. Наприклад, число 23.5141 має  precision 6 та scale 4. Цілочисленні значення можна вважати числами з масштабом 0.

        Для стовпчика типу numeric можна налаштувати максимальну precision і максимальний scale. Стовпець типу numeric оголошується так:

   ```SQL
   NUMERIC(precision, scale)

   NUMERIC(precision)      # scale=0

   NUMERIC                 # створює стовпець типу «необмежену кількість» , у якому можна зберігати числові значення
                           # будь-якої довжини межі, обумовленого реалізацією. У стовпчику цього типу вхідні значення не
                           # будуть наводитися до будь-якого масштабу, тоді як у стовпцях numericіз явно заданим масштабом
                           # значення підганяють під цей масштаб
   ```

    3. [Числа з плаваючою комою](https://www.postgresql.org/docs/14/datatype-numeric.html#DATATYPE-FLOAT).
        Типи даних real,  double precision зберігають наближені числові значення зі змінною точністю. Відзначимо лише таке
            - якщо вам потрібна точність при зберіганні та обчисленнях (наприклад, для грошових сум), використовуйте натомість тип numeric.
            - якщо ви хочете виконувати з цими типами складні обчислення, що мають велику важливість, ретельно вивчіть реалізацію операцій у вашому середовищі і особливо поведінку в крайніх випадках (нескінченність, антипереповнення).
            - перевірка рівності двох чисел з плаваючою точкою може завжди давати очікуваний результат.
    4. [Послідовні типи](https://www.postgresql.org/docs/14/datatype-numeric.html#DATATYPE-SERIAL).
        Ці типи даних - smallserial, serial, bigserial - не зовсім типи - це зручний спосіб створити стовпці з унікальними ідентифікаторами. Простіше їх розуміти як створення генератора послідовності, який на кожному кроці повертає нове значення (навіть якщо в результаті, за якоїсь причини, рядок в таблиці почав створюватись і не створився (наприклад - відкат транзакції) то число яке було повернуто для цього рядка при намаганні його створити вже ніколи не буде використано). Тобто - генератор віддає значення один раз і більше ніколи.

2. __Символьні типи__ ([докладно тут](https://www.postgresql.org/docs/14/datatype-character.html)).

    Ім'я | Опис
    -----|-----
    character varying(n),varchar(n) | рядок обмеженої змінної довжини
    character(n), char(n) | рядок фіксованої довжини, доповнений пробілами
    text | рядок необмеженої змінної довжини

    SQL визначає декілька основних символьних типів:
    - character varying(n) (синонім - varchar(n)) - текстовий рядок довжиною до __n__ символів (саме символів, не байт!) Спроба зберегти в стовпці такого типу довший рядок приведе до помилки, якщо всі зайві символи не є пробілами (тоді вони будуть усічені до максимально допустимої довжини). (Це дещо дивний виняток продиктований стандартом SQL). Якщо довжина рядка, що зберігається, виявляється меншою за оголошену - просто збереже короткий рядок.
    - character(n) (синонім char(n)) - все те ж саме що і попередній тип, але зберігається завжди рівно __n__ символів - якщо ввести менше то доповнюється пробілами.

    Якщо задано __n__, воно має бути більше нуля і менше або дорівнює 10 485 760. Якщо ж довжина не вказується для типа, цей тип прийматиме рядки будь-якого розміру.

    Крім цього, PostgreSQL пропонує тип text, у якому можна зберігати рядки довільної довжини. Хоча тип text не описаний у стандарті SQL, його підтримують деякі інші СУБД SQL.

3. __Типи дати\часу__
    PostgreSQL підтримує повний набір типів дати та часу SQL. Операції, можливі з цими типами даних, [описані тут](https://www.postgresql.org/docs/14/functions-datetime.html). Усі дати вважаються за Григоріанським календарем, навіть для часу до його введення.

    Ім'я | Розмір  | Опис | Найменше значення | Найбільше значення | Точність
    -----|---------|------|-------------------|--------------------|----------
    timestamp [ (p) ] [ without time zone ] | 8 байт  | дата та час (без часового поясу) | 4713 р. до н. е. | 294276 н. е. | 1 мікросекунда
    timestamp [ (p) ] with time zone | 8 байт  | дата та час (з часовим поясом) | 4713 р. до н. е. | 294276 н. е. | 1 мікросекунда
    date | 4 байти | дата (без доби) | 4713 р. до н. е. | 5874897 н. е. | 1 день
    time [ (p) ] [ without time zone ] | 8 байт  | час доби (без дати) | 00:00:00 | 24:00:00 | 1 мікросекунда
    time [ (p) ] with time zone | 12 байт | час дня (без дати), з часовим поясом | 00:00:00+1559 | 24:00:00-1559 | 1 мікросекунда
    interval [ поля ] [ (p) ] | 16 байт | часовий інтервал | -178000000 років | 178000000 років | 1 мікросекунд


4. __Логічні типи__
    У PostgreSQL є стандартний SQL - тип boolean. Тип boolean може мати такі стани: "true", "false" і третій стан, "unknown", яке представляється SQL -значенням NULL.

    Ім'я | Розмір | Опис
    -----|--------|-----
    boolean | 1 байт | стан: true чи false


Цей перелік типів даних далеко не повний і не в повній мірі описує поведінку і можливості перелічених типів даних. PostgreSQL описує грошові типи даних (money), двійкові типи (bytea), типи перерахувань (enum), геометричні типи даних (точки, прямі, відрізки, прямокутники, шляхи, багатокутники, окружності), типи мережевих адрес, є також типи орієнтовані на текстовий пошук (tsvector, tsquery), UUID-тип, XML-тип, JSON-тип, масиви, диапазонні типи і це не повний перелік.

Я сподіваюсь що ви побачили що PostgreSQL має значну кількість вбудованих типів даних і можливість створювати свої типи. При виникненні необхідності - читайте документацію і знаходьте рішення. Такий простір варінтів точно дає можливість для цього.

### Прості запити.

Ми з Вами вже робили такі запити при першому знайомстві з SQL і БД SQLite, але для узагальнення і повторення пройдемо по основним з невеликим розширенням переліку запитів.

Ми будемо говорити про теорію (якщо це буде необхідно), а потім описувати запит для виконпння з нашого коду і демонструвати цей запит за допомогою графічного інтерфейсу якогось з підключених нами клієнтів.

#### SELECT

[докладно тут](https://www.postgresql.org/docs/14/sql-select.html)


##### Повна виборка

In [ ]:
orders_full_data_sample = """
SELECT * FROM orders;
"""

In [ ]:
orders_full_sample = execute_read_query(connection, orders_full_data_sample)
print(orders_full_sample)

У фінальному наборі ми отримали всі рядки цієї таблиці і всі атрибути. Це знанадто багато даних і, як правило, такої потреби немає. Фільтрувати по рядкам - ми будемо далі, а для того щоб фільтрувати по атрибутам - просто вкажіть необхідні атрибути таблиці замість зірочки через кому:

#### Виборка конкретного набора атрибутів

In [ ]:
orders_some_attributes_sample = """
SELECT order_id, order_date, shipped_date, ship_city
FROM orders;
"""
orders_partly_sample = execute_read_query(connection, orders_some_attributes_sample)
print(orders_partly_sample)

#### Прості математичні операції та фукції

[опис тут](https://www.postgresql.org/docs/14/functions.html)

- додавання         "+"
- віднімання        "-"
- множення          "*"
- поділ             "/" (якщо оператори цілі числа - результат ціле число - округляється в напрямку нуля)
- зміна знака       "- (числовий тип)"
- залишок від цілочисленого ділення - "%"
- зведення в ступінь "numeric ^ numeric" | "double precision ^ double precision"
- квадратний корінь "|/ double precision"
- кубічний корінь "||/ double precision"
- абсолютне значення "@ (числовий тип)"

Операцій багато - дивіться документацію.

Приклад використання:
у нас є таблиця products з атрибутами unit_price - вартість одиниці продукцї і units_in_stock - залишок на складі. Давайте виведемо назву товару і суму вартості товару цієї назви на складі:

In [ ]:
operation_query = """
SELECT product_name, unit_price * units_in_stock
FROM products;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

PostgreSQL надає велику кількість [вбудованих функцій](https://www.postgresql.org/docs/14/functions.html).
Ми не будемо вникати в кожну, подивимось декілька прикладів використання.

Знайти найбільший спільний дільник:

In [ ]:
operation_query = """
SELECT gcd(1028, 12004);
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Повернути довільне число (в діапазоні 0.0 <= x < 1.0):

In [ ]:
operation_query = """
SELECT random();
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### DISTINCT

вибрати унікальні рядки

(наприклад - ми хочемо мати список країн в яких живуть зараз наші покупці (таблиця customers))

ВАЖЛИВО! Якщо ми вибираємо декілька атрибутів - то у виборку попадуть лише унікальні комбінації

In [ ]:
operation_query = """
SELECT DISTINCT country
FROM customers;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

In [ ]:
operation_query = """
SELECT DISTINCT country, city
FROM customers;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### COUNT

підрахунок кількості отриманих у виборці рядків

важливо розуміти, що COUNT лише рахує рядки. Якщо необхідно підрахувати унікальні рядки - необхідно комбінувати з DISTINCT

Підрахуємо кількість наших постачальників

In [ ]:
operation_query = """
SELECT COUNT(*)
FROM suppliers;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

А от якщо ми захочемо підрахувати країни в яких працюють всі наші працівники (таблиця employees), то запит буде більш складний:

In [ ]:
operation_query = """
SELECT COUNT(DISTINCT country)
FROM employees;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### WHERE

фільтуємо дані по умові

Формат:
```SQL
SELECT <*>
FROM <table_name>
WHERE <condition>;
```

condition ---> boolean result

В фінальний набір попадають рядки, для яких умова буде мати значення true

Наприклад, можливі умови (можуть бути й інші):
- a = b
- a > b
- a >= b
- a < b
- a <= b
- a != b

Давайте отримаємо всіх постачальників в якійсь конкретній країні:

In [ ]:
operation_query = """
SELECT company_name, city, phone
FROM suppliers
WHERE country = 'USA';
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### AND, OR

комбінація умов

```SQL
SELECT <*>
FROM <table_name>
WHERE <condition_1> AND|OR <condition_2>;
```

Давайте розширимо наш попередній запит і отримаємо постачальників не тільки з США, а й з Германії:

In [ ]:
operation_query = """
SELECT company_name, city, phone, country
FROM suppliers
WHERE country = 'USA' OR country = 'Germany';
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### BETWEEN

Важливо пам'ятати - границі включають в інтервал, тобто це еквівалент
age BETWEEN 30 AND 40 еквівалентно age >= 30 AND age <= 40

Давайте зробимо запит який потребує знання декількох вже вивчених пунктів. Наприклад - вважаємо що наша HR-служба компанії запросила інформацію про всіх співробітників яки мають вік від 25 до 35 років і працюють з нами більше двох років.
Треба згадати:
- у нас є таблиця employees в якій є атрибути age_date і hire_date - дата народження і дата найма в компанію
- є типи даних - data (викристовуються для ціх атрибутів) і interval - [докладно тут](https://www.postgresql.org/docs/14/datatype-datetime.html)
- можливо нам необхідно буде визначити поточну дату або - різницю поточної дати і якоїсь іншої. [Згадаємо вбудовані функції](https://www.postgresql.org/docs/14/functions-datetime.html).

Якщо все скласти, отримаємо:

In [ ]:
operation_query = """
SELECT employee_id, last_name, first_name, city, age(birth_date) AS age, age(hire_date) AS time_in_compaty
from employees
where (age(birth_date) between  interval '45 years' and interval '60 years') and (age(hire_date) > interval '5 years');
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Я використав невідомий нам ще оператор AS. І до того ж, ми бачимо у виводі наші інтервали не дуже зручно - все в днях. Необхідно розуміти, що це відповідь для нас сформована в об'єктах python. Пропоную Вам перенести цей запит в pgAdmin або DBeaver і подивитись там не результат.
Якщо, все ж таки, роль оператора AS лишається незрозумілою - [документація тут](https://www.postgresql.org/docs/14/sql-select.html).

#### IN, NOT IN

Пам'ятаєте ми нещодавно робили запит, в якому хотіли отримати всіх постачальників з Германії і США. Давайте перепишемо цей зпит по іншому:

In [ ]:
operation_query = """
SELECT company_name, city, phone, country
FROM suppliers
WHERE country IN ('USA','Germany');
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Для мене такий формат здається більш читабельним. Думаю, що з "антогоністом" цього запиту - __NOT IN__ - ви розберетесь.

#### ORDER BY

впорядкування результатів

ASC (за замовченням) - за зростанням
DESC - за зменшенням

Можливо сортування за декількома атрибутами

Давайте отримаємо список всіх наших покупців відсортований по країнах:

In [ ]:
operation_query = """
SELECT company_name, country, city
FROM customers
ORDER BY country;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Щоб зрозуміти - як це сортування по декількох полях, давайте відсортуємо не тільки по країні, но і по місту.

In [ ]:
operation_query = """
SELECT company_name, country, city
FROM customers
ORDER BY country DESC, city DESC;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### MIN, MAX, AVG

Агрегаційні вирази - [докладно тут](https://www.postgresql.org/docs/14/sql-expressions.html#SYNTAX-AGGREGATES)

Агрегатний вираз є застосування агрегатної функції до рядків, обраним запитом. Агрегатна функція зводить безліч вхідних значень одного вихідного, як наприклад, сума або середнє або мінімальне\максимальне значення.

Знайдемо, наприклад, середню вартість товарів у нас в наявності:

In [ ]:
operation_query = """
SELECT AVG(unit_price)
FROM products;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Або - дату самого першого замовлення з дроставкою в Германію:

In [ ]:
operation_query = """
SELECT MIN(order_date)
FROM orders
WHERE ship_country = 'Germany';
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

Або - дату народження самого молодого співробітника компанії який працює у Лондоні:

In [ ]:
operation_query = """
SELECT MAX(birth_date)
FROM employees
WHERE city = 'London';
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### LIKE

відповідність шаблону

SQL надає нам два символа підстановки:

% - означає 0, 1 або більше будь яких символів

_ - означає один будь який символ

Приклад:
- LIKE 'AA%' - рядки, які починаються з двох символів A
- LIKE '%AA' - рядки, які закінчуються двома символами A
- LIKE '%Марія%' - рядки які включають в собі субрядок 'Марія'
- LIKE 'www._%.__' - рядок, який починається з www. далі один, або більше символів до крапки (але хочаб один), крапка і два будь-яких символа в кінці

Наприклад - знайдемо всіх співробітників ім'я яких починається з "L":

In [ ]:
operation_query = """
SELECT employee_id, first_name, last_name, city
FROM employees
WHERE first_name LIKE 'L%';
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### LIMIT

Якщо Вам необхідно обмежити виборку (уявіть що у Вас БД з петабайтами даних і сотнями тисяч\мільонами записів в таблицях) - ви можете використати LIMIT:

In [ ]:
operation_query = """
SELECT employee_id, first_name, last_name, city
FROM employees
ORDER BY first_name
LIMIT 10;
"""

final_set = execute_read_query(connection, operation_query)
print(final_set)

#### Check on NULL

NULL -спеціальне значення, яке визначає не 0 (як число), а відсутність будь-якого значення. При мапінгу цього значення на об'єкти python це буде None. Наприклад - давайте подивимось на атрибут ship_region таблиці orders:

In [ ]:
operation_query = """
SELECT order_id, order_date, ship_region, ship_name, ship_city
FROM orders
LIMIT 20;
"""

final_set = execute_read_query(connection, operation_query)
for record in final_set:
    print(record)

Напевно - це був не обов'язковий атрибут.
Давайте відфільтруємо записи так, щоб отримати записи для яких цей запис існує:

In [ ]:
operation_query = """
SELECT order_id, order_date, ship_region, ship_name, ship_city
FROM orders
WHERE ship_region IS NOT NULL
LIMIT 20;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

#### GROUP BY

Допомагає сгрупувати результати.
Простіше пояснити на прикладі: нам необхідно побачити всі замовлення з вагою більше 40 кг і щоб вони були згруповані по країнам.

In [ ]:
operation_query = """
SELECT ship_country,  COUNT(*) 
FROM orders
WHERE freight >= 40
GROUP BY ship_country
ORDER BY COUNT(*) DESC;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Або інший приклад - у нас є категоріх товарів (products.category_id) і самі товари (які відносяться до якихось певних категорій). Також - у нас є для кожного товару його залишки на складі - products.units_in_stock.
Давайте отримаємо таблицю яка надасть нам інформацію про кількість одиниць товарів в кожній категорії:

In [ ]:
operation_query = """
SELECT category_id,  SUM(units_in_stock) 
FROM products
GROUP BY category_id
ORDER BY SUM(units_in_stock) DESC;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

#### HAVING

Ми з вами вже багато раз фільтували наш запит - використовуючи WHERE.
Тепер уявіть ситуацію що ви використали WHERE і потім згрупували результат за допомогою GROUP BY, і Вам необхідно накласти ще якійсь фільтр на результат. Для цього існує оператор HAVING.

Нам потрібно оцінити суму на яку продається товарів в категорії товару і вивести ті - яких в категорії залишилось небагато - зкладські залишки менше 10000.
Зроуміло що нам необхідно згрупувати товари по category_id, а потім порахувати суму SUM(units_in_price * units_in_stock). Сгрупувати результати по категоріям і додатково відфільтрувати - за допомогою HAVING:

In [ ]:
operation_query = """
SELECT category_id,  SUM(units_in_stock * unit_price) 
FROM products
GROUP BY category_id
HAVING SUM(units_in_stock * unit_price) < 10000;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

HAVING працює точно так як і WHERE, але з результатами первинної фільтрації.

#### UNION, INTERSECT, EXCEPT

Ці оператори повинні викликати у Вас деяке відчуття déjà vu. І це абсолютно справедливе відчуття - коли ми з вами вивчали множини, ми стикалися з операціями над множинами - поєднання множин, перетин множин, різні варіанти різниць множин.
Тепер ми вивчимо операції над множинами в SQL і PostgreSQL. І - нагадаю вам, що всі таблиці (або відношення) - це є множини, результат будь-якого запиту - це також множина (або таблиця, або відношення).

Давайте виведемо всі країни з яких наші працівники і з яких наші постачальники. Але це дві різні таблиці - тому робити це треба за допомогою поєднання двох множин за допомогою UNION.

In [ ]:
operation_query = """
SELECT country
FROM suppliers
UNION
SELECT country
FROM employees
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Як ви бачите - дублікатів нема. Якщо Вам для чогось потрібні повтри країн - використовуйте UNION ALL.

INTERSECT - перетин множин.
Наприклад - давайте подивимось на країни, де у нас є і постачальники і покупці (можливо ми фантазуємо як оптимізувати доставку і не використовувати свох склади):

In [ ]:
operation_query = """
SELECT country
FROM suppliers
INTERSECT
SELECT country
FROM customers
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Давайте тепер з'ясуємо в яких країнах у нас є покупці, але немає взагалі ніяких постачальніків.
Тобто - нам необхідно отримати множину всіх країн, де є покупці, а потім вилучити звідти всі країни, де є наші постачальники. Така операція робиться з допомогою EXCEPT:

In [ ]:
operation_query = """
SELECT country
FROM customers
EXCEPT
SELECT country
FROM suppliers
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Як ви бачите EXCEPT контролює унікальність - тобто кожна країна зустрічається у фінальній виборці лише один раз.
Існує конструкція EXCEPT ALL - яка, фактично, враховує кількість повторів в першій множині в другій. І починає в результаті залишати різницю - тобто вважає кожен елемент унікальним і вираховує симетричну різніцю. 
Пропоную Вам самостійно поексперементувати з цім оператором.